In [1]:
!pip install selenium --quiet
!pip install bs4

In [2]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [3]:
# bs scrapt data
root = "https://www.politico.com/2022-election/results/"
url = "https://www.politico.com/2022-election/results/"
response = requests.get(url, timeout=3)
soup = bs(response.content, 'html.parser')

# states list
a_elements = soup.select('div.styles_columns-container__Kq5vS div a')
states_list = [state.text for state in a_elements]

In [4]:
# state code
code_dic = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [5]:
# init selenium
from selenium import webdriver
from selenium.common.exceptions import TimeoutException,NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

chrome_options = webdriver.ChromeOptions()
chrome_options.page_load_strategy = 'eager'
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.headless = True

In [6]:
# scrap
States = []
States_code = []
Districts = []
Party = []
Candidates = []
Incumbent = []
Vote = []
Pct = []

#test_list = ["New Jersey"]

driver = webdriver.Chrome(options=chrome_options)

for state in states_list:
  link = root + state.lower().replace(' ','-') + "/house/"
  driver.get(link)
  #  click footer expand first, then click all expand
  try:
      footer_button = driver.find_element(By.CSS_SELECTOR, 'div.styles_footer-container__mcmjY button[title="Expand"]')
      footer_button.click()
  except NoSuchElementException:
    pass

  try:
      buttons = driver.find_elements(By.CSS_SELECTOR, 'button[title="Expand"]')
      for button in buttons:
        if button == footer_button:
          continue
        print("found button")
        driver.execute_script("window.scrollTo(0,0);")
        button.click()

  except NoSuchElementException:
    pass

  #district data
  dist = driver.find_elements(By.CSS_SELECTOR,'div.styles_table-container__vTHda > div')
  for i in dist:
    try:
      dist_number = i.find_element(By.CSS_SELECTOR, 'h5.styles_is-table__D_lzx.styles_is-desktop__4Cq7X').text
    except NoSuchElementException:
      dist_number = "1st"
      pass
    cddts = i.find_elements(By.CSS_SELECTOR,'table > tbody > tr.styles_container__sXqdi')
    for j in cddts:
        cddt_data = j.find_element(By.CSS_SELECTOR, 'span:nth-last-child(1)').text
        data = cddt_data.split(' (')
        # State and State code
        States.append(state)
        States_code.append(code_dic[state])

        # District
        Districts.append(dist_number)

        # Party
        if len(data) > 1:
          if data[1] == "R)":
            pt = "Republican"
          elif data[1] == "D)":
            pt = "Democratic"
          elif data[1] == "Libertarian Party)":
            pt = "Libertarian Party"
          else:
            pt = "other"
        else:
          pt = "no party"
        Party.append(pt)

        # Candidate name
        cddt = data[0].replace("*","")
        Candidates.append(cddt)

        # Incumbent
        icbt =   "Incumbent" if '*' in cddt_data else ""
        Incumbent.append(icbt)

        # Percent & Vote
        #TODO Uncontested = 100%
        vt = '0'
        p = j.find_element(By.CSS_SELECTOR,'td:nth-last-child(1)').text
        vt = j.find_element(By.CSS_SELECTOR,'td:nth-last-child(2)').get_attribute('textContent').replace(',','')
        vt = 0 if p=="Uncontested" else int(vt)
        p = "100%" if p=="Uncontested" else p
        Pct.append(p)
        Vote.append(vt)

        print(f"state={state}, dis={dist_number},cddt={cddt},party={pt} icbt={icbt}, vt={vt}, p={p}")
driver.quit()

state=Alabama, dis=1st,cddt=Carl,party=Republican icbt=Incumbent, vt=139854, p=84.2%
state=Alabama, dis=1st,cddt=Remrey,party=Libertarian Party icbt=, vt=26197, p=15.8%
state=Alabama, dis=2nd,cddt=Moore,party=Republican icbt=Incumbent, vt=137193, p=69.2%
state=Alabama, dis=2nd,cddt=Harvey-Hall,party=Democratic icbt=, vt=57800, p=29.1%
state=Alabama, dis=2nd,cddt=Realz,party=Libertarian Party icbt=, vt=3380, p=1.7%
state=Alabama, dis=3rd,cddt=Rogers,party=Republican icbt=Incumbent, vt=135448, p=71.3%
state=Alabama, dis=3rd,cddt=Veasey,party=Democratic icbt=, vt=47770, p=25.1%
state=Alabama, dis=3rd,cddt=Bell,party=no party icbt=, vt=3826, p=2.0%
state=Alabama, dis=3rd,cddt=Casson,party=Libertarian Party icbt=, vt=3027, p=1.6%
state=Alabama, dis=4th,cddt=Aderholt,party=Republican icbt=Incumbent, vt=164371, p=84.2%
state=Alabama, dis=4th,cddt=Neighbors,party=Democratic icbt=, vt=26628, p=13.6%
state=Alabama, dis=4th,cddt=Cochran,party=Libertarian Party icbt=, vt=4294, p=2.2%
state=Alabama

In [10]:
# data frame

data = {
 'State': States,
 'State Code': States_code,
 'District': Districts,
 'Party': Party,
 'Candidate': Candidates,
 'Incumbent': Incumbent,
 'Vote': Vote,
 'Pct%': Pct
}
df = pd.DataFrame(data)

condition = (df['State'] == 'Indiana') & (df['District'] == '2nd')
rows_to_delete = df[condition].head(3).index
dframe = df.drop(rows_to_delete).reset_index(drop=True)

dframe.to_csv('election_results.csv', index=False)
dframe

,State,State Code,District,Party,Candidate,Incumbent,Vote,Pct%
0,Alabama,AL,1st,Republican,Carl,Incumbent,139854,84.2%
1,Alabama,AL,1st,Libertarian Party,Remrey,,26197,15.8%
2,Alabama,AL,2nd,Republican,Moore,Incumbent,137193,69.2%
3,Alabama,AL,2nd,Democratic,Harvey-Hall,,57800,29.1%
4,Alabama,AL,2nd,Libertarian Party,Realz,,3380,1.7%
...,...,...,...,...,...,...,...,...
1052,Wisconsin,WI,8th,Libertarian Party,VandenPlas,,32043,10.5%
1053,Wyoming,WY,1st,Republican,Harriet Hageman,,132215,69.8%
1054,Wyoming,WY,1st,Democratic,Lynnette GreyBull,,47241,24.9%
1055,Wyoming,WY,1st,Libertarian Party,Richard Brubaker,,5420,2.9%
